# Funciones: MLOps

In [2]:
# Importamos las librerias correspondientes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Funcion 1: developer()
+ def **developer( *`desarrollador` : str* )**:
    `Cantidad` de items y `porcentaje` de contenido Free por año según empresa desarrolladora. 
Ejemplo de retorno:

| Año  | Cantidad de Items | Contenido Free  |
|------|-------------------|------------------|
| 2023 | 50                | 27%              |
| 2022 | 45                | 25%              |
| xxxx | xx                | xx%              |

Importación de datos

In [25]:
# Importamos el DataFrame a archivo .parquet
df_developer= pd.read_parquet('steam_games.parquet')

In [26]:
# Se eliminan las variables que no ayudan a la formación de la funcion
df_developer.drop(columns=['publisher','app_name','price','average price'],inplace=True)

# Se realiza el cambio de formato de fecha sin contar con los errores
df_developer['release_date'] = pd.to_datetime(df_developer['release_date'],format='%Y-%m-%d', errors='coerce')

# Se crea otra columna con los años de la columna con formato fecha
df_developer['year']= df_developer.release_date.dt.year

# agregamos una columna que me haga conocer si hay FREE TO PLAY
df_developer['free']= df_developer['genres'].apply(lambda x: 'yes' if 'Free to Play' in x else 'no')

# Eliminamos genres y release_date por no ser necesarios al df de la funcion
df_developer.drop(columns=['genres','release_date'],inplace=True)

In [27]:
df_developer.head()

,id,developer,year,free
index,,,,
0,761140,Kotoshiro,2018.0,no
1,643980,Secret Level SRL,2018.0,yes
3,767400,彼岸领域,2017.0,no
4,772540,Trickjump Games Ltd,2018.0,no
5,774276,Poppermost Productions,2018.0,yes


Exportación del dataset para la funcion 1

In [28]:
df_developer.to_parquet('df1_developer.parquet')

Función

In [29]:
def developer(desarrollador:str):
    # EXPLORATORY DATA ANALYSIS (EDA)
    # importamos los valores
    developer= pd.read_parquet('df1_developer.parquet')


    # obtenemos el dataframe segun el desarrollador
    developer= developer[developer['developer'] == desarrollador]

    # La cantidad de entregas que hay por año
    games_anio_ordenado= developer.groupby('year')['id'].count().sort_index(ascending=False)
    games_anio_ordenado= games_anio_ordenado.reset_index()  # se convierte los indices en columnas 
    games_anio_ordenado.columns= ['Año','Cantidad de Items']  # cambiamos el nombre de las columnas



    # realizamos la agrupacion y contamos los elemnentos por año y por ser 'Free to Play' 
    contenido_free= developer.groupby('year')['free'].value_counts(normalize=True).sort_index(ascending=False)*100
    contenido_free= contenido_free.reset_index()  # Se colocan los indices como columnas
    contenido_free= contenido_free[contenido_free['free'] == 'yes']  # Se realiza el filtro con solo los 'yes'
    contenido_free.columns= ['Año','yes_no','Contenido Free']  # Se cambia el nombre de las columnas
    

    # realizamos el juntar la tabla de las cantidad de items y porcentaje de contenido Free
    # tambien se realizan otras modificaciones al llenar con ceros los 'NaN', eliminacion de la columna 'yes_no'
    resultado= pd.merge(games_anio_ordenado, contenido_free, on='Año', how='left')
    resultado['Contenido Free']= resultado['Contenido Free'].fillna(0)
    resultado['Contenido Free']= resultado['Contenido Free'].apply(lambda x: str(round(x,2)) + '%')
    resultado['Año']= resultado['Año'].apply(lambda x: int(x))
    resultado.drop(columns= ['yes_no'], inplace= True)
    
    return resultado

In [33]:
developer('Valve')

,Año,Cantidad de Items,Contenido Free
0,2017,2,50.0%
1,2016,1,0.0%
2,2012,1,0.0%
3,2011,1,0.0%
4,2010,2,0.0%
5,2009,1,0.0%
6,2008,1,0.0%
7,2007,3,33.33%
8,2006,2,0.0%
9,2004,5,0.0%


In [42]:
developer('Valve').to_dict(orient='records')

[{'Año': 2017, 'Cantidad de Items': 2, 'Contenido Free': '50.0%'},
 {'Año': 2016, 'Cantidad de Items': 1, 'Contenido Free': '0.0%'},
 {'Año': 2012, 'Cantidad de Items': 1, 'Contenido Free': '0.0%'},
 {'Año': 2011, 'Cantidad de Items': 1, 'Contenido Free': '0.0%'},
 {'Año': 2010, 'Cantidad de Items': 2, 'Contenido Free': '0.0%'},
 {'Año': 2009, 'Cantidad de Items': 1, 'Contenido Free': '0.0%'},
 {'Año': 2008, 'Cantidad de Items': 1, 'Contenido Free': '0.0%'},
 {'Año': 2007, 'Cantidad de Items': 3, 'Contenido Free': '33.33%'},
 {'Año': 2006, 'Cantidad de Items': 2, 'Contenido Free': '0.0%'},
 {'Año': 2004, 'Cantidad de Items': 5, 'Contenido Free': '0.0%'},
 {'Año': 2003, 'Cantidad de Items': 1, 'Contenido Free': '0.0%'},
 {'Año': 2001, 'Cantidad de Items': 1, 'Contenido Free': '0.0%'},
 {'Año': 2000, 'Cantidad de Items': 2, 'Contenido Free': '0.0%'},
 {'Año': 1999, 'Cantidad de Items': 1, 'Contenido Free': '0.0%'},
 {'Año': 1998, 'Cantidad de Items': 1, 'Contenido Free': '0.0%'}]

In [5]:
# Se toma otra variable con el nombre del desarrollador
games_anio_desa= games_anio[games_anio['developer'] == 'Valve']

In [6]:
# La cantidad de entregas que hay por año
games_anio_ordenado= games.groupby('year')['app_name'].count().sort_index(ascending=False)
games_anio_ordenado= games_anio_ordenado.reset_index()
games_anio_ordenado.columns= ['Año','Cantidad de Items']

In [8]:
contenido_free= games_anio_desa.groupby('year')['free'].value_counts(normalize=True).sort_index(ascending=False)*100
contenido_free= contenido_free.reset_index()
contenido_free= contenido_free[contenido_free['free'] == 'yes']
contenido_free.columns= ['Año','yes_no','Contenido Free']

In [9]:
resultado= pd.merge(games_anio_ordenado, contenido_free, on='Año', how='left')
resultado['Contenido Free']= resultado['Contenido Free'].fillna(0)
resultado['Contenido Free']= resultado['Contenido Free'].apply(lambda x: str(round(x,2)) + '%')
resultado['Año']= resultado['Año'].apply(lambda x: int(x))
resultado.drop(columns= ['yes_no'], inplace= True)

In [10]:
resultado

,Año,Cantidad de Items,Contenido Free
0,2021,1,0.0%
1,2019,1,0.0%
2,2018,67,0.0%
3,2017,8972,50.0%
4,2016,6455,0.0%
5,2015,4401,0.0%
6,2014,2499,0.0%
7,2013,1408,0.0%
8,2012,1096,0.0%
9,2011,524,0.0%


## Función 2: userdata
+ def **userdata( *`User_id` : str* )**:
    Debe devolver `cantidad` de dinero gastado por el usuario, el `porcentaje` de recomendación en base a reviews.recommend y `cantidad de items`.

Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}